In [ ]:
# https://pypng.readthedocs.io/en/latest/ca.html
!pip install pypng

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pydicom
import png
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage.transform import resize
import tensorflow as tf
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
train_dir = "/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/"
test_dir = "/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/test/"

train_images = os.listdir(train_dir)
test_images = os.listdir(test_dir)

train_path_image = [train_dir + image for image in train_images]
test_path_image = [test_dir + image for image in test_images]

In [ ]:
filetrain = './trainpng/' 

if not os.path.exists(os.path.dirname(filetrain)):
    os.makedirs(filetrain)
    
filetest = './testpng/' 

if not os.path.exists(os.path.dirname(filetest)):
    os.makedirs(filetest)

In [ ]:
filetrain + train_path_image[0].split('/')[-1].replace(".dicom", ".png")

In [ ]:
def convert_to_png(full_path_image, filename):
    print(filename)
    for dicomimage in tqdm(full_path_image):
        ds = pydicom.dcmread(dicomimage)
        img = apply_voi_lut(ds.pixel_array, ds)
        img = resize(img, (1024, 1024), anti_aliasing=True)
        
        if ds.PhotometricInterpretation == "MONOCHROME1":
            img = np.amax(img) - img
            
        img = (((img - np.min(img))/np.max(img))*255.0).astype(np.uint8) 
        
        with open(filename + dicomimage.split('/')[-1].replace(".dicom", ".png"), "wb") as fn:
            
            w = png.Writer(1024, 1024, greyscale=True)
            w.write(fn, img)
       

In [ ]:
convert_to_png(train_path_image[:100], filetrain)

In [ ]:
convert_to_png(test_path_image[:100], filetest)

In [ ]:
!tar -zcf train.tar.gz './trainpng'
!tar -zcf test.tar.gz './testpng'